In [ ]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
# Local imports
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml defaults_cfg 


experiment:
    exp_root: '?'
    threshold: 0.5 
    inference_seed: 40

log:
    root: '?'
    min_fg_pixels: 1 
    log_interval: 10 
    log_image_stats: True 
    log_pixel_stats: False 
    gether_inference_stats: True
    summary_compute_global_metrics: False 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

In [ ]:
%%yaml calibration_cfg 

calibrator:
    _name: '?'

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: '?' 
    num_prob_bins: 15
    neighborhood_width: 3

In [ ]:
%%yaml model_cfg 

# For standard datasets
#####################################
data:
    preload: False 

model:
    _type: "standard"
    pretrained_exp_root : None
    pretrained_select_metric: "val-dice_score"
    checkpoint: '?'
    pred_label: '?' 

## Gather Inference Options.

In [ ]:
%%yaml experiment_cfg 

name: '06_17_24_EXPANDED_DRIVE_Hard'

base_models: "/storage/vbutoi/scratch/ESE/training/06_18_24_WMH_Large_WMH_HardLabels"

data:
    _class: "ese.data.datasets.WMH"
    annotator: "observer_o12"
    task: ("Amsterdam", "Singapore")
    label_threshold: null 
    split: ("train", "val", "cal")

In [ ]:
# Get the configs for the different runs.
base_cfg = Config(defaults_cfg).update([calibration_cfg, model_cfg])
# Get the different experiment cfg yamls.
inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    # run_name=exp_name,
    show_examples=True,
    gpu='1',
) 

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
# )